In [1]:
import pandas as pd
from src.training.longformer_pipeline import LongFormerPipeline
from src.training.bert_pipeline import TrainingBertPipeline
import logging
import torch
import os

In [2]:
df = pd.read_csv("data/aes_dataset_5k_clean.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5363 entries, 0 to 5362
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   question          4859 non-null   object 
 1   reference_answer  5363 non-null   object 
 2   answer            5363 non-null   object 
 3   score             5363 non-null   float64
 4   normalized_score  5363 non-null   float64
 5   multibert_length  5363 non-null   int64  
 6   dataset           5363 non-null   object 
 7   dataset_num       5363 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 335.3+ KB


In [3]:
# Check if the first file exists
df_result = None
if os.path.exists("experiments/results/results_longformer.csv"):
    df_result = pd.read_csv("experiments/results/results_longformer.csv")
    print(df_result['config_id'].iloc[-1])
else:
    print("File 'results_longformer.csv' does not exist.")

4


In [4]:
batch_sizes = [4]
epochs_list = [5]
learning_rates = [1e-5, 2e-5, 5e-5]
idx = (df_result['config_id'].iloc[-1] + 1) if df_result is not None and not df_result.empty else 0  # index untuk setiap kombinasi
ROOT_DIR = os.getcwd()

In [5]:
for batch_size in batch_sizes:
    for num_epochs in epochs_list:
        for lr in learning_rates:
            results = []
            results_epoch = []
            df_result1 = None
            # Check if the second file exists
            if os.path.exists("experiments/results/results_epoch_longformer.csv"):
                df_result1 = pd.read_csv("experiments/results/results_epoch_longformer.csv")
                print(max(df_result1['valid_qwk']))
            else:
                print("File 'results_epoch_longformer.csv' does not exist.")
                
            config = {
                "df": df,
                "model_name": "allenai/longformer-base-4096",
                "batch_size": batch_size,
                "learning_rate": lr,
                "epochs": num_epochs,
                "config_id": idx,
                "max_seq_len": 2048,
                "best_valid_qwk": max(df_result1['valid_qwk']) if df_result1 is not None and not df_result1.empty else float("-inf"),
            }

            logging.info(
                f"Running configuration: config_id={idx}, model_name={config['model_name']}, "
                f"batch_size={batch_size}, epochs={num_epochs}, learning_rate={lr}, max_seq={config['max_seq_len']}"
            )
            print(
                f"\nRunning configuration: config_id={idx}, model_name={config['model_name']}, "
                f"batch_size={batch_size}, epochs={num_epochs}, learning_rate={lr}, max_seq={config['max_seq_len']}"
            )
            
            try:
                pipeline = LongFormerPipeline(config, results, results_epoch)
                pipeline.run_training()

                # Save results
                # Dapatkan root project
                results_path = os.path.join(ROOT_DIR, "experiments/results/results_longformer.csv")
                results_epoch_path = os.path.join(ROOT_DIR, "experiments/results/results_epoch_longformer.csv")
                TrainingBertPipeline.save_csv(results, results_path)
                TrainingBertPipeline.save_csv(results_epoch, results_epoch_path)
            except Exception as e:
                logging.error(f"Error in config_id={idx}: {str(e)}")
                print(f"Error in config_id={idx}: {str(e)}")
                torch.cuda.empty_cache()
            finally:
                # Clear GPU memory after every configuration
                del pipeline.model
                del pipeline.tokenizer
                del pipeline.optimizer
                torch.cuda.empty_cache()
            idx += 1

0.8936687654885577

Running configuration: config_id=5, model_name=allenai/longformer-base-4096, batch_size=2, epochs=10, learning_rate=5e-05, max_seq=2048
split dataset run...
create dataset run...
create dataloader run...
create dataloader done...
====== Training Epoch 1/10 ======
Train Loss: 0.1226, Train QWK: 0.0928, Train Pearson: 0.1166
Validation Loss: 0.1144, Validation QWK: 0.0000, Validation Pearson: 0.6263
====== Training Epoch 2/10 ======
Train Loss: 0.1181, Train QWK: -0.0040, Train Pearson: -0.0095
Validation Loss: 0.1151, Validation QWK: 0.0000, Validation Pearson: -0.0002
====== Training Epoch 3/10 ======
Train Loss: 0.1144, Train QWK: -0.0160, Train Pearson: -0.0130
Validation Loss: 0.1131, Validation QWK: 0.0000, Validation Pearson: 0.6050
====== Training Epoch 4/10 ======
Train Loss: 0.1111, Train QWK: -0.0068, Train Pearson: 0.0371
Validation Loss: 0.1389, Validation QWK: 0.0000, Validation Pearson: 0.3440
====== Training Epoch 5/10 ======
Train Loss: 0.1054, Train 